In [1]:
!pip install gTTS

In [2]:
!pip install -U openai opencv-python moviepy requests imageio[ffmpeg]

In [14]:
import cv2
import base64
from openai import OpenAI
import os
from google.colab import userdata
import re

# API key from alibaba cloud
api_key = userdata.get('DASHSCOPE_API_KEY')

client = OpenAI(
    api_key=api_key,
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
)

def process_frames(video_path, fps_sample_rate=1):
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    if fps != 0:
        video_length_seconds = length / fps
        print(f'Video length: {video_length_seconds:.2f} seconds')
    else:
        raise ValueError("Video not found or unreadable.")

    base64_frames = []
    frame_interval = int(fps * fps_sample_rate)
    frame_index = 0

    while video.isOpened():
        success, frame = video.read()
        if not success:
            break
        if frame_index % frame_interval == 0:
            _, buffer = cv2.imencode(".jpg", frame)
            base64_image = base64.b64encode(buffer).decode("utf-8")
            base64_frames.append(base64_image)
        frame_index += 1

    video.release()
    print(f"{len(base64_frames)} frames sampled.")
    return base64_frames, video_length_seconds

def create_continuous_prompt(video_length_seconds):
    return f"""
    Generate continuous Arabic football commentary for these video frames.

    Team and Player Information:
    - Bayern Munich (Red jerseys): 7 Serge Gnabry, 9 Harry Kane, 10 Leroy Sane
    - Borussia Dortmund (White jerseys): #9 Sebastian Haller, #11 Marco Reus

    Requirements for Natural Speech Output:
    1. Use only Arabic commentary style
    2. Focus on fluid play-by-play narration
    3. Avoid bullet points, lists, or structured sections
    4. Use natural transitions between actions
    5. Include appropriate Arabic football commentary expressions
    6. Maintain consistent energy throughout
    7. Target duration: {video_length_seconds:.1f} seconds when spoken

    Commentary Style Guidelines:
    - Use connecting phrases like "والآن", "نرى", "يحاول"
    - Mention either the player's name or jersey number, not both
    - Include emotional reactions when appropriate
    - Keep sentences concise but flowing
    - Avoid technical analysis interruptions
    """

video_path = "/content/test_video.mp4"
base64_frames, video_length = process_frames(video_path)

image_messages = [
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{img}"
        }
    } for img in base64_frames[:80]
]


messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": create_continuous_prompt(video_length)}
        ] + image_messages
    }
]

response = client.chat.completions.create(
    model="qwen2.5-vl-32b-instruct",
    messages=messages
)

final_commentary = response.choices[0].message.content
print(final_commentary)

Video length: 30.00 seconds
30 frames sampled.
اللعبة تبدأ بضغط كبير من بايرن ميونخ. لاعبو البايرن يسيطرون على وسط الملعب،  ويحاولون بناء الهجمات بشكل سريع. هاري كين يتحرك بحرية في منطقة جزاء دورتموند،  بينما سيرج غنابري يراقب الدفاع عن قرب. "هاري كين يتلقى كرة طويلة من خط الوسط،  لكن دفاع دورتموند يتدخل بسرعة ويقطعها." الكرة تنتقل إلى لوروا ساني الذي يحاول التقدم نحو منطقة الجزاء،  لكنه يواجه ضغطًا كبيرًا من لاعبي دورتموند. "ساني يمرر الكرة إلى غنابري،  الذي يحاول التسديد،  لكن الحارس يتصدى للكرة ببراعة." دورتموند يحاول استعادة السيطرة على المباراة،  مع محاولة ماركو ريوس إيجاد فرصة للهجوم. "ريوس يمرر الكرة إلى سيباستيان هالر،  لكن الدفاع يقطعها قبل أن يصل إليها." اللعبة تستمر بالتبادل السريع بين الهجمات والدفاعات،  مع سيطرة نسبية من بايرن ميونخ على الكرة. "بايرن ميونخ يحافظ على الكرة بحذر،  ويحاول إيجاد الفرصة المناسبة للتسجيل." في هذه الأثناء،  الجماهير تتابع المباراة بحماس شديد،  وتنتظر أي هجمة قد تؤدي إلى هدف. "بايرن ميونخ يشن هجوماً معاكساً،  لكن دفاع دورتموند ينجح في إبعاد الخطر.

In [9]:
import re
from gtts import gTTS
def clean_vl_output_for_tts(text):
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    text = re.sub(r'^\d+\.\s*', '', text, flags=re.MULTILINE)
    text = re.sub(r'-\s*', '', text)
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text).strip()
    text = re.sub(r'[^\u0600-\u06FF\s،؛؟]+', '', text)
    return text

cleaned = clean_vl_output_for_tts(final_commentary)


tts = gTTS(text=cleaned, lang='ar')
tts.save("commentary.mp3")
